# M2 | Submission: Wage Regression

Course: Gen Bus 656: Foundations of Statistical Learning for Business Analytics\
Author: Matthew Brown\
Date: October 4, 2025

## Submission Instructions

1. In the course GitHub repository, locate the file `'GB886_II_9_Wages_1985_Current_Population_Survey.csv'`. This data set is a cross-sectional sample from the May 1985 Current Population Survey by the US Census Bureau. 
2. These data include (hourly) wages for **534 individuals**, where we have information on: 
    + age
    + sex (0 for male, 1 for female)
    + race (H for Hispanic, W for White, O for Other)
    + years of education
    + et cetera
3. Set up a Codebook that loads this datafile.
4. Run a linear regression model that uses `Wage' as the outcome/target/dependent variable and all the available features/independent variables as variables (you will have to convert the categorical variables into dummy variables).
5. Produce a regression table to include:
    + R-squared
    + RMSE
    + MAE
    + MAPE
6. Discuss conclusions from your exercise to include: 
    + What are viable use cases and take-aways
    + What are potentially problematic conclusions one shouldn't take away
7. Submit your work for review via a sharable link to your Colab Codebook.

## Load Necessary Libraries

In [1]:
# Load necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

## Load Data into Dataframe

In [3]:
# Clone into the course github repository
!git clone https://github.com/danielbauer1979/MSDIA_PredictiveModelingAndMachineLearning.git

Cloning into 'MSDIA_PredictiveModelingAndMachineLearning'...


In [4]:
# Load data from csv file in the repository into wages dataframe if coming from github repo.
wages_df = pd.read_csv('MSDIA_PredictiveModelingAndMachineLearning/GB886_II_9_Wages_1985_Current_Population_Survey.csv')
wages_df.head()

,Yrs_Ed,Sthrn_Rgn,Sex,Yrs_Exprnc,Union,Wage,Age,Race,Occup,Sect,Marr
0,8,0,1,21,0,5.10,35,H,Other,Manf,1
1,9,0,1,42,0,4.95,57,W,Other,Manf,1
2,12,0,0,1,0,6.67,19,W,Other,Manf,0
3,12,0,0,4,0,4.00,22,W,Other,Other,0
4,12,0,0,17,0,7.50,35,W,Other,Other,1


In [ ]:
# Load data from csv file into wages dataframe if coming from local drive.
#wages_df = pd.read_csv('../Data/GB886_II_9_Wages_1985_Current_Population_Survey.csv')
#wages_df.head()

,Yrs_Ed,Sthrn_Rgn,Sex,Yrs_Exprnc,Union,Wage,Age,Race,Occup,Sect,Marr
0,8,0,1,21,0,5.10,35,H,Other,Manf,1
1,9,0,1,42,0,4.95,57,W,Other,Manf,1
2,12,0,0,1,0,6.67,19,W,Other,Manf,0
3,12,0,0,4,0,4.00,22,W,Other,Other,0
4,12,0,0,17,0,7.50,35,W,Other,Other,1


## Prepare Data for Model

## Run Linear Regression Model

### Train Linear Regression Model

### Calculate Required Metrics

### Discuss Findings and Conclusions